In [1]:
import numpy as np
import pandas as pd 



In [3]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'tmdb_5000_movies.csv'

In [ ]:
movies.head().shape

In [ ]:
credits.head().shape

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.info()

In [ ]:
movies.shape

In [ ]:
movies = movies.drop(['budget','homepage','original_language',
       'original_title','popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime','vote_average',
       'vote_count','spoken_languages', 'status', 'tagline','id'],axis=1)

In [ ]:
movies.head()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.iloc[0].genres # movies.genres[0] - we can also do this to get every columns data

In [ ]:
import ast
#ast.literal_eval()----- this function will change string to list

In [ ]:
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
def convertcast(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i['name'])
            counter += 1
        else:
            break
    return l

In [ ]:
movies['cast'] = movies['cast'].apply(convertcast)#in this we get name of every starting 3 main actors name 

In [ ]:
def director(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l

In [ ]:
movies['crew'] = movies['crew'].apply(director)

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
 movies['tags']= movies['genres'] + movies['keywords'] + movies['overview'] + movies['cast'] + movies['crew']

In [ ]:
df = movies.drop(['genres','keywords','overview','cast','crew'],axis=1)

In [ ]:
df['tags'] = df['tags'].apply(lambda x: " ".join(x)) # with these we can convert list to string 
df['tags'][1]

In [ ]:
#recommended ---covert every words in lower case
df['tags'] = df['tags'].apply(lambda x:x.lower())

In [ ]:
import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    l=[]
    
    for i in text.split():
        l.append(ps.stem(i))
    return " ".join(l)

In [ ]:
df['tags'] = df['tags'].apply(stem)

In [ ]:
# now i do some text prepocesing such as nlp- bag of words , removal opf stop words

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features = 5000,stop_words = 'english')

In [ ]:
 vectors = cv.fit_transform(df['tags']).toarray() # by default count vector will return sparse matix so we convet it to numpy array 

In [ ]:
vectors.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
df.head(1)

In [ ]:
def recommend(movie):
    movie_index = df[df['title'] == movie].index[0] #it will fetch the index value
    distance = similarity[movie_index]
    movies_list = sorted(list(enumerate(distance)),reverse = True,key =lambda x:x[1])[1:6]
    for i in movies_list:
        print(df.loc[i[0]].title)

In [ ]:
recommend('Avatar')

In [ ]:
import pickle

In [ ]:
pickle.dump(df,open('movies.pkl','wb'))

In [ ]:
#pickle.dump(df.to_dict,open('movies_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))